In [1]:
import pandas as pd
import sqlalchemy as sqla

from pymongo import MongoClient

In [2]:
reviews = pd.read_csv('../src/reviews.csv')

In [3]:
def create_rev_df(df, optional_cols=[], positive=0, ren_cols={}):
    # Columns that need to be used everytime
    mandatory_cols = ['Hotel_Name', 'lat', 'lng', 'Review_Total_Negative_Word_Counts', 'Average_Score']
    
    # get the most intersting columns from the dataframe
    _cols = optional_cols + mandatory_cols
    the_df = df.loc[:, _cols]
    
    the_df['positive'] = positive
    
    # This rename is mandatory and is all kinds of reviews
    mandatory_ren = {
        'Hotel_Name': 'name',
        'Review_Total_Negative_Word_Counts': 'word_count',
        'Average_Score': 'score'
    }
    
    # Merge dictonary and rename the columsn
    ren_cols.update(mandatory_ren)
    the_df = the_df.rename(columns=ren_cols)
    
    # Return only the reviews with content
    return the_df[the_df.word_count != 0]

In [4]:
neg_df = create_rev_df(reviews, ['Negative_Review'], 0, {'Negative_Review': 'review'})

In [5]:
pos_df = create_rev_df(reviews, ['Positive_Review'], 1, {'Positive_Review': 'review'})

In [6]:
reviews_df = pd.concat([neg_df, pos_df]).reset_index(drop=True)

In [7]:
client = MongoClient("localhost:27017")
db=client.hotels

In [8]:
db.reviews.insert_many(reviews_df.to_dict('records'))